In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
writer = SummaryWriter()

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [19]:
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/encoded.csv", nrows=10000)
#df.loc[400:,"enc_feature_0":"enc_feature_49"]
#strip = 1 + df.shape[0] % 200
#df.loc[:df.shape[0]-strip]
#df["enc_feature_0", "feature_0"]
df.insert(3, "trade", None)
df.loc[df["resp"] <= 0, "trade"] = -1
df.loc[df["resp"] > 0, "trade"] = 1
#df = df[df["weight"] > 0]
df["weight"].min()
df.loc[df["resp"] * df["resp_4"] < 0]

,Unnamed: 0,date,weight,trade,resp_1,resp_2,resp_3,resp_4,resp,feature_0,...,enc_feature_40,enc_feature_41,enc_feature_42,enc_feature_43,enc_feature_44,enc_feature_45,enc_feature_46,enc_feature_47,enc_feature_48,enc_feature_49
34,34,0.0,119.137190,-1,-0.003248,-0.003826,-0.001878,0.000884,-0.001324,-1.0,...,-4.601709,-3.751104,1.383849,0.449079,-3.503184,-1.172947,-1.274178,-0.807073,0.696374,-0.092873
35,35,0.0,0.000000,-1,-0.003097,-0.003713,-0.001231,0.000930,-0.001171,-1.0,...,-1.057161,-1.566746,-0.813210,-2.755063,-0.124268,-1.186755,0.138651,-0.892173,-1.822678,0.047349
36,36,0.0,0.000000,-1,-0.002654,-0.003300,-0.000663,0.001892,-0.000911,-1.0,...,-1.317378,-1.735831,-0.401609,-2.375719,-0.146330,-1.170636,0.059221,-0.628190,-1.867367,-0.062876
37,37,0.0,1.150897,-1,-0.002724,-0.003229,0.000046,0.001423,-0.000789,-1.0,...,-0.880958,-1.575679,-0.626259,-2.737603,0.126620,-1.329390,0.247901,-0.890701,-1.560826,0.121249
38,38,0.0,1.109739,-1,-0.002159,-0.003411,-0.000987,0.001227,-0.000835,-1.0,...,1.963213,-1.666897,1.345630,-1.060685,2.867054,1.817773,2.019053,-0.933096,0.900719,3.142029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9971,1.0,5.322012,1,0.019857,0.021749,0.007583,-0.004495,0.015877,-1.0,...,1.015480,-2.524155,1.733027,1.310184,-0.085147,-2.861717,-2.909841,-2.134443,-0.599073,-1.450151
9980,9980,1.0,4.259218,-1,-0.007781,-0.012479,-0.000859,0.002630,-0.014959,1.0,...,1.087621,-1.781044,-1.408776,-0.192932,-2.852556,-0.970918,-1.476856,0.437832,1.527355,1.624339
9981,9981,1.0,1.266787,1,0.000781,0.000538,-0.000087,-0.000531,0.000524,-1.0,...,-0.528907,-0.742144,-0.213862,-1.272695,-1.302219,-0.534337,-2.457932,-1.514828,0.162272,-1.802363
9984,9984,1.0,0.027776,-1,0.005035,0.004189,0.006692,0.003811,-0.000138,1.0,...,1.281844,-0.726377,-1.258649,-1.857394,-1.006484,1.470896,-2.421484,-0.759736,-0.404747,1.540813


In [4]:
t = torch.tensor([1.0,2000.0,3.0])
noise = torch.randn_like(t)
noise.multiply(1)

tensor([-0.4874, -1.1828, -0.3263])

In [5]:
VALIDATION_SIZE = 1000
p=0.1
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/train.csv", nrows=10000, skiprows=lambda i: i > 0 and np.random.random() > p)


In [6]:
df = df.fillna(0)
df.loc[df['resp'] <= 0, 'trade'] = 0
df.loc[df['resp'] > 0, 'trade'] = 1

In [7]:
feature_0_pos = df.columns.get_loc("feature_0")
df.insert(feature_0_pos, "w_date", None)
df.insert(feature_0_pos, "w_ts_id", None)
df.loc[:, "w_date"] = (df["date"] / df["date"].max()) * 10
for one in range(df["date"].max()+1):
    min_id = df.loc[df["date"] == one, :]["ts_id"].min()
    max_id = df.loc[df["date"] == one, :]["ts_id"].max()
    distance = max_id - min_id
    df.loc[df["date"] == one, "w_ts_id"] = (df.loc[df["date"] == one, :]["ts_id"] - min_id) / distance

df.iloc[4000]["w_ts_id"]

0.7366515837104073

In [8]:
class JaneStreetDataset(Dataset):

    # Constructor with defult values
    def __init__(self, df, transform=None):
        self.df = df
        self.len = len(self.df)
        self.transform = transform

    # Getter
    def __getitem__(self, index):
        sample = torch.from_numpy(self.df.iloc[index]["feature_0":"feature_129"].values), torch.tensor(self.df.iloc[index]["resp"])
        sample = sample[0].to(device), sample[1].to(device)
        if self.transform:
            sample = self.transform(sample)
        return sample

    # Get Length
    def __len__(self):
        return self.len

In [9]:
df.iloc[2]["resp"]

0.010657450355787305

In [10]:
df.iloc[2]["feature_0":"feature_129"].values

array([1, 0.4309994855102589, -0.56656447366909, 0.9173147504036292,
       0.4740209628811886, 0.8680641224473489, 0.4848058293948703, 0.0,
       0.0, 0.9304045758114388, -0.12719137318286966, 0.0, 0.0,
       2.1797353711169336, 0.9885256898461467, 1.1614696571391283,
       0.3814523209688711, 0.0, 0.0, 2.054515587190643,
       1.5186244781980016, 0.0, 0.0, 3.01550530462673, 2.591359696948621,
       2.3744079683277852, 1.691290967949599, 0.0, 0.0,
       -0.1112465826145948, -0.1255963088203609, 0.0, 0.0,
       -0.015873817169229976, -0.014315091291192081, -0.08335632835247217,
       -0.08377188780165852, 1.157410408097431, 0.7625436514408253,
       0.009306697715007223, 0.018458686324766358, 0.5370037693713132,
       -0.9167707744600301, 2.261787963568694, 0.7483851622022146,
       1.8319323046102087, -0.549480978941606, -0.8081752907322435,
       -1.0176971520189724, -1.0484148632280508, -0.43194249486048575,
       -0.8297540630649565, -1.6932307001919789, 0.195233569208

In [11]:
DROPOUT_P = 0.0
big_number = 1000
small_number = 500
model = torch.nn.Sequential(
    torch.nn.Linear(130, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, 1),
)
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)


<ipython-input-11-ae5219358239>:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [12]:
janestreet_train = JaneStreetDataset(df=df.iloc[:-VALIDATION_SIZE])
janestreet_validation = JaneStreetDataset(df=df.iloc[-VALIDATION_SIZE:])
BATCH_SIZE = 100
N_EPOCHS = 50
LR = 0.05
train_loader = torch.utils.data.DataLoader(dataset=janestreet_train, batch_size=BATCH_SIZE, drop_last=True)
validation_loader = torch.utils.data.DataLoader(dataset=janestreet_validation, batch_size=BATCH_SIZE, drop_last=True)

In [13]:
model = model.float()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.001)
criterion = nn.MSELoss()

In [14]:
def train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=None):
    accuracy_list = []
    loss_list = []
    for epoch in range(n_epochs):
        for x, y in train_loader:
            model.train()
            optimizer.zero_grad()
            z = model(x.float())
            y = y.view(-1, 1).float()
            loss = criterion(z, y)
            writer.add_scalar("Train data", loss, epoch)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.data)
        # perform a prediction on the validation data
        for x_test, y_test in validation_loader:
            model.eval()
            z = model(x_test.float())
            y = y.view(-1, 1)
            loss = criterion(z, y.float())
            writer.add_scalar("Validation data", loss, epoch)
            accuracy_list.append(loss.data)
    return accuracy_list, loss_list

In [15]:
accuracy_list, loss_list = train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=N_EPOCHS)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
model.eval()

In [ ]:
index=2000
#a = torch.from_numpy(df.iloc[index]["feature_0":"feature_129"].values)
a = df.iloc[index]["feature_0":"feature_129"].values
resp = torch.tensor(df.iloc[index]["resp"])

r = []
for one in range(BATCH_SIZE):
    r.append(a)
r = np.array(r)
model_input =torch.from_numpy(r).float().to(device)
z = model(model_input)

(resp, z[0])

In [ ]:
accuracy_list

In [ ]:
loss_list